<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b3a80ad1bbdf425d3d69e0deffef8e51a83b0ef670399d4e052f3f612fab5be7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-28 14:14:03
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -38.26 K (-0.27%)
Current PnL: -22.77 L (-14.92%)
CY Booked + Current PnL: -8.88 L (-5.82%)
-------------------
Total profit:  1.57 L
Total loss:  -24.34 L
-------------------
Total Booked + Current PnL: 18.10 L (14.4%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.71%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.19 L (61.66%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.35%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.0,0.36,-8.69,9.54,0.02,15206.0,-15169.0,159390.0,143.30,67.0,M-SC,10.46,234.0,-1.00,1.11,38.64,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.0,-2.72,-16.46,19.81,0.09,16679.0,-16583.0,84194.0,95.52,38.0,M-SC,3.49,253.0,-0.99,0.59,10.61,OX40N,NTT,DURABLES
43,ITC,452.0,-0.27,-1.49,12.10,10.44,23857.0,-2973.0,197165.0,-41.15,44.0,X-LC,1.21,5.0,-0.12,1.38,4.00,X40,NTT,FMCG
83,VOLTAS,1530.0,-2.31,6.84,12.03,19.69,24644.0,13113.0,204855.0,-3.56,48.0,X-MC,3.30,78.0,0.53,1.43,14.10,XY25,NTT,AC
52,MEDANTA,1486.0,-0.60,3.75,19.60,24.08,25814.0,4755.0,131705.0,-4.69,47.0,X-SC,5.88,89.0,0.18,0.92,24.10,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,3.15,-2.64,39.34,35.67,120977.0,-8326.0,307516.0,-14.21,62.0,X-LC,5.72,4.0,-0.07,2.15,10.85,X40N,ATH,FMCG
14,CAMPUS,393.00,2.08,-6.11,41.95,33.28,66083.0,-10247.0,157528.0,-24.34,54.0,M-SC,2.93,222.0,-0.16,1.10,22.09,XY24,NTT,FOOTWEAR
51,MASFIN,398.61,1.74,-5.17,28.71,22.05,26674.0,-5070.0,92910.0,-18.25,51.0,H-SC,7.01,164.0,-0.19,0.65,35.05,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,1.44,-13.52,110.36,81.92,109562.0,-15521.0,99277.0,-49.35,26.0,M-SC,2.90,236.0,-0.14,0.69,17.53,XR,NTT,DURABLES
47,KANSAINER,340.00,1.16,-20.99,43.62,13.47,92935.0,-56610.0,213057.0,-67.53,39.0,H-SC,1.47,158.0,-0.61,1.49,8.33,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIS,528.0,-4.02,-24.00,59.56,21.26,50257.0,-26652.0,84380.0,1990.67,47.0,H-SC,3.87,166.0,-0.53,0.59,14.01,OX40N,NTT,MISC
58,RAJOOENG,143.1,-3.69,-27.59,92.81,39.61,68884.0,-28280.0,74220.0,-51.04,35.0,H-SC,21.14,135.0,-0.41,0.52,4.61,AR,ATH,MISC
78,TTKPRESTIG,770.0,-2.72,-16.46,19.81,0.09,16679.0,-16583.0,84194.0,95.52,38.0,M-SC,3.49,253.0,-0.99,0.59,10.61,OX40N,NTT,DURABLES
6,ASIANTILES,137.0,-2.32,-14.07,111.42,81.67,89817.0,-13199.0,80611.0,7244.44,47.0,L-SC,19.48,271.0,-0.15,0.56,56.11,XR,NTT,CERAMICS
83,VOLTAS,1530.0,-2.31,6.84,12.03,19.69,24644.0,13113.0,204855.0,-3.56,48.0,X-MC,3.30,78.0,0.53,1.43,14.10,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.37,0.20,106.31,106.72,131383.0,249.0,123585.0,-50.44,39.0,H-SC,2.78,139.0,0.00,0.86,23.63,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.50,-0.67,36.55,35.63,76082.0,-1414.0,208160.0,-14.53,44.0,H-MC,3.77,119.0,-0.02,1.46,15.88,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.48,-5.00,118.86,107.91,166712.0,-7385.0,140259.0,-22.58,35.0,M-SC,10.67,216.0,-0.04,0.98,0.29,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.36,-8.69,9.54,0.02,15206.0,-15169.0,159390.0,143.30,67.0,M-SC,10.46,234.0,-1.00,1.11,38.64,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-2.72,-16.46,19.81,0.09,16679.0,-16583.0,84194.0,95.52,38.0,M-SC,3.49,253.0,-0.99,0.59,10.61,OX40N,NTT,DURABLES
47,KANSAINER,340.00,1.16,-20.99,43.62,13.47,92935.0,-56610.0,213057.0,-67.53,39.0,H-SC,1.47,158.0,-0.61,1.49,8.33,XY24,NTT,PAINTS
67,SIS,528.00,-4.02,-24.00,59.56,21.26,50257.0,-26652.0,84380.0,1990.67,47.0,H-SC,3.87,166.0,-0.53,0.59,14.01,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.02,2.34,68.33,72.27,118490.0,3961.0,173409.0,-11.98,59.0,M-LC,3.69,99.0,0.03,1.21,8.66,XR,NTT,IT
38,INDIAMART,4810.62,0.37,0.20,106.31,106.72,131383.0,249.0,123585.0,-50.44,39.0,H-SC,2.78,139.0,0.00,0.86,23.63,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.02,2.34,68.33,72.27,118490.0,3961.0,173409.0,-11.98,59.0,M-LC,3.69,99.0,0.03,1.21,8.66,XR,NTT,IT
38,INDIAMART,4810.62,0.37,0.20,106.31,106.72,131383.0,249.0,123585.0,-50.44,39.0,H-SC,2.78,139.0,0.00,0.86,23.63,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.50,-0.67,36.55,35.63,76082.0,-1414.0,208160.0,-14.53,44.0,H-MC,3.77,119.0,-0.02,1.46,15.88,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.48,-5.00,118.86,107.91,166712.0,-7385.0,140259.0,-22.58,35.0,M-SC,10.67,216.0,-0.04,0.98,0.29,OX40N,ATH,CABLES
1,ABB,7934.00,-1.17,-4.98,53.20,45.57,132251.0,-13027.0,248592.0,-38.95,54.0,H-MC,5.77,120.0,-0.10,1.74,5.78,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.06,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.11,-15.31,67.88,42.18,158905.0,-42313.0,234097.0,-24.30,28.0,X-LC,1.94,3.0,-0.27,1.64,1.40,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.43,-8.70,26.20,15.23,47926.0,-17422.0,182925.0,-21.89,31.0,X-MC,6.96,66.0,-0.36,1.28,20.75,X40N,NTT,REALTY
9,BAJAJHFL,181.50,-0.28,-14.07,72.76,48.45,125364.0,-28208.0,172298.0,-20.30,31.0,X-MC,7.54,64.0,-0.23,1.20,0.10,X40N,ATH,FINANCE
55,PGHH,17907.65,0.15,-4.40,39.91,33.76,76621.0,-8835.0,191985.0,-32.51,34.0,X-MC,3.87,57.0,-0.12,1.34,0.57,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.27,-1.49,12.10,10.44,23857.0,-2973.0,197165.0,-41.15,44.0,X-LC,1.21,5.0,-0.12,1.38,4.00,X40,NTT,FMCG
3,ACC,3906.0,-1.18,-22.33,111.34,64.15,205779.0,-53131.0,184820.0,-54.57,48.0,X-SC,1.58,82.0,-0.26,1.29,3.67,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.54,1.50,22.94,24.79,58643.0,3782.0,255638.0,-3.25,48.0,X-SC,1.86,86.0,0.06,1.79,24.99,X40N,NTT,MISC
66,SIEMENS,4671.5,-0.76,-11.68,42.12,25.51,69224.0,-21745.0,164350.0,0.93,62.0,H-LC,1.87,49.0,-0.31,1.15,21.11,AR,ATH,ELECTRICAL
76,TMPV,600.0,-0.11,-15.31,67.88,42.18,158905.0,-42313.0,234097.0,-24.30,28.0,X-LC,1.94,3.0,-0.27,1.64,1.40,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.06,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-1.35,-6.18,34.37,26.07,52799.0,-10120.0,153620.0,-30.92,34.0,X-MC,9.57,55.0,-0.19,1.07,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.28,-14.07,72.76,48.45,125364.0,-28208.0,172298.0,-20.30,31.0,X-MC,7.54,64.0,-0.23,1.20,0.10,X40N,ATH,FINANCE
55,PGHH,17907.65,0.15,-4.40,39.91,33.76,76621.0,-8835.0,191985.0,-32.51,34.0,X-MC,3.87,57.0,-0.12,1.34,0.57,X40,ATH,FMCG
76,TMPV,600.00,-0.11,-15.31,67.88,42.18,158905.0,-42313.0,234097.0,-24.30,28.0,X-LC,1.94,3.0,-0.27,1.64,1.40,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.38,-27.74,97.68,42.84,45883.0,-18033.0,46973.0,-52.62,42.0,X-SC,37.57,83.0,-0.39,0.33,2.21,XY24,NTT,MISC
60,RELAXO,1176.00,-0.28,-47.03,187.53,52.31,144196.0,-68268.0,76892.0,-43.57,37.0,X-SC,4.52,91.0,-0.47,0.54,2.92,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.06,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.44,-6.38,29.87,21.58,39190.0,-8937.0,131202.0,-52.80,35.0,X-MC,6.42,56.0,-0.23,0.92,16.16,X40N,ATH,IT
52,MEDANTA,1486.00,-0.60,3.75,19.60,24.08,25814.0,4755.0,131705.0,-4.69,47.0,X-SC,5.88,89.0,0.18,0.92,24.10,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.14,-14.66,39.77,19.28,117419.0,-50711.0,295245.0,-25.57,59.0,X-LC,7.11,1.0,-0.43,2.06,9.15,X40,ATH,IT
41,INFY,2275.00,-0.13,6.50,45.42,54.87,151347.0,20329.0,333217.0,-17.09,62.0,X-LC,2.47,2.0,0.13,2.33,15.32,X40,BTT,IT
76,TMPV,600.00,-0.11,-15.31,67.88,42.18,158905.0,-42313.0,234097.0,-24.30,28.0,X-LC,1.94,3.0,-0.27,1.64,1.40,XY24,NTT,AUTO
82,VBL,671.64,3.15,-2.64,39.34,35.67,120977.0,-8326.0,307516.0,-14.21,62.0,X-LC,5.72,4.0,-0.07,2.15,10.85,X40N,ATH,FMCG
43,ITC,452.00,-0.27,-1.49,12.10,10.44,23857.0,-2973.0,197165.0,-41.15,44.0,X-LC,1.21,5.0,-0.12,1.38,4.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.28,-36.87,110.51,32.89,53860.0,-28468.0,48738.0,-701.92,64.0,L-MC,5.75,259.0,-0.53,0.34,34.22,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.32,-14.07,111.42,81.67,89817.0,-13199.0,80611.0,7244.44,47.0,L-SC,19.48,271.0,-0.15,0.56,56.11,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.12,-11.75,58.13,39.54,51891.0,-11891.0,89268.0,-31.03,67.0,M-SC,6.67,220.0,-0.23,0.62,25.88,AR,ATH,AUTO
51,MASFIN,398.61,1.74,-5.17,28.71,22.05,26674.0,-5070.0,92910.0,-18.25,51.0,H-SC,7.01,164.0,-0.19,0.65,35.05,XR,ATH,FINANCE
70,SURYODAY,216.00,0.97,-19.12,52.11,23.03,75476.0,-34231.0,144840.0,57.89,50.0,H-SC,9.59,167.0,-0.45,1.01,43.71,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.36,-8.69,9.54,0.02,15206.0,-15169.0,159390.0,143.30,67.0,M-SC,10.46,234.0,-1.00,1.11,38.64,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.16,0.47,20.38,20.95,51551.0,1179.0,252947.0,-0.49,74.0,X-LC,15.17,31.0,0.02,1.77,36.27,X40,ATH,PAINTS
50,LTIM,7201.88,1.10,11.36,18.23,31.66,49972.0,27967.0,274118.0,7.16,67.0,H-LC,13.59,43.0,0.56,1.92,52.10,X200,ATH,IT
79,UNITDSPR,1644.00,0.66,6.91,12.96,20.77,32443.0,16185.0,250329.0,-2.32,63.0,X-MC,3.79,62.0,0.50,1.75,14.31,X40N,NTT,BREWERIES
32,HCLTECH,1922.01,0.25,5.98,17.70,24.73,45379.0,14457.0,256381.0,10.64,64.0,X-LC,4.79,7.0,0.32,1.79,22.12,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.94
1,25,44.87
2,50,75.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.42
MC    29.16
LC    25.39
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.82
X40      21.23
X40N     12.01
XR        9.51
XY25      9.08
AR        9.06
OX40N     7.60
X200      1.92
SR        0.98
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.86
X-MC    22.67
X-LC    20.14
M-SC    11.92
X-SC     8.24
H-MC     4.76
H-LC     3.07
L-SC     1.40
M-MC     1.39
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.85,-4.37,38.12
IT,13.82,-14.37,73.42
FINANCE,9.49,-14.07,62.27
MISC,7.33,-23.85,74.58
ELECTRICAL,5.98,-9.67,49.84
PAINTS,5.96,-8.22,24.29
INSURANCE,4.41,-0.37,35.74
PHARMA,3.90,-0.43,32.58
AUTO,3.37,-19.06,70.11


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3056063.0,21
XR,1299747.0,13
AR,1276342.0,10
X40,1001607.0,14
X40N,731346.0,9
OX40N,705973.0,10
XY25,341392.0,6
SR,280838.0,2
MH,75203.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3564665.0,25
M-SC,1353399.0,15
X-MC,1239063.0,16
X-LC,881680.0,11
X-SC,741062.0,8
H-MC,398592.0,3
L-SC,261289.0,3
H-LC,119196.0,2
M-LC,118490.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1210651.0      6
           AR         878111.0      5
           XR         795521.0      7
M-SC       XY24       780506.0      6
X-MC       X40        444101.0      7
X-LC       X40        389553.0      5
X-MC       XY24       369089.0      3
H-SC       OX40N      324341.0      4
X-SC       X40N       295633.0      3
M-SC       OX40N      293777.0      5
H-SC       SR         280838.0      2
X-SC       XY24       277476.0      3
X-MC       X40N       244923.0      4
X-LC       XY24       228082.0      2
H-MC       AR         208333.0      2
X-LC       X40N       190790.0      2
H-MC       XY24       190259.0      1
X-MC       XY25       180950.0      2
L-SC       XR         173434.0      2
X-SC       X40        167953.0      2
M-SC       XR         158442.0      2
           AR         120674.0      2
M-LC       XR         118490.0      1
L-SC       OX40N       87855.0      1
H-SC       MH          75203.0      1
X-LC       XY25        73255.0      2
H-LC       AR          69224.0      1
L-MC       XR          53860.0      1
H-LC       X200        49972.0      1
M-MC       XY25        46645.0      1
L-LC       XY25        40542.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
